# Hybrid LCA
### "you mean it's that easy?"

Hybrid LCA has a certain mystique about it but it's really just the same-- the only difference is that things are 
measured in dollars instead of units.  In a graph-based modeling framework, mixing process-based and EEIO data is
trivial.

In [1]:
from antelope_foreground import ForegroundCatalog
from antelope import enum

In [2]:
cat = ForegroundCatalog()
cat.blackbook_authenticate('https://sc.vault.lc', 'bkuczenski')


Loading JSON data from /home/b/size/lib/python3.11/site-packages/antelope_core/archives/data/elcd_reference_quantities.json:
local.qdb: /home/b/size/lib/python3.11/site-packages/antelope_core/archives/data/elcd_reference_quantities.json
local.qdb: Setting NSUUID (False) 77833297-6780-49bf-a61a-0cb707dce700
local.qdb: /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json
28 total quantity entities added (28 new)
6 total flow entities added (6 new)


Enter password to access blackbook server at https://sc.vault.lc:  ········


POST https://sc.vault.lc/auth/token.. 200 [0.52 sec]
Welcome back to blackbook, hosted by ANTELOPE_AUTHORITY.
Username: bkuczenski, email: bkuczenski@ucsb.edu
Last login: Mon Sep 23 04:34:03 2024 
token expires in 36000 s


Our approach here will be to find a USLCI process with a cutoff flow, and patch it with USEEIO

In [3]:
r_uslci = cat.get_blackbook_resources('lcacommons.uslci.fy24.q1.01')
r_useeio = cat.get_blackbook_resources('lcacommons.useeio.2.0.1')
r_traci = cat.get_blackbook_resources('lcia.traci.2.1')

GET https://sc.vault.lc/origins/lcacommons.uslci.fy24.q1.01/resource.. 200 [0.14 sec]
GET https://sc.vault.lc/origins/lcacommons.useeio.2.0.1/resource.. 200 [0.29 sec]
GET https://sc.vault.lc/origins/lcia.traci.2.1/resource.. 200 [0.13 sec]


# Part 1 - Modeling Vinyl Ester Resin 
First we find the foreground process that describes our product

In [4]:
q_uslci = cat.query(r_uslci[0].origin)

QQQQQQQQQQQQQQQQQQ lcacommons.uslci.fy24.q1.01 QQQQQQQQQQQQQQQQQQ
Found Antelope providers:
AntelopeMeta:_dev
antelope_core.providers:IlcdArchive
antelope_core.providers:IlcdLcia
antelope_core.providers:EcospoldV2Archive
antelope_core.providers:EcospoldV1Archive
antelope_core.providers:EcoinventLcia
antelope_core.providers:OpenLcaJsonLdArchive
antelope_core.providers:Traci21Factors
antelope_core.providers:XdbClient
antelope_core.providers:OpenLcaRefData
antelope_background.providers:TarjanBackground
antelope_background.providers:Background
antelope_foreground.providers:AntelopeV1Client
antelope_foreground.providers:LcForeground
antelope_foreground.providers:OryxClient
GET https://bk.vault.lc/origins.. 200 [0.18 sec]
lcacommons.uslci.fy24.q1.01: https://bk.vault.lc/
lcacommons.uslci.fy24.q1.01: Setting NSUUID (False) None
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/config.. 200 [0.05 sec]
Applying configuration to XdbClient with 0 entities at https://bk.vault.lc/


In [5]:
pvcs = enum(q_uslci.flows(name='vinyl'))

GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/flows.. 200 [0.10 sec]
 [00] [lcacommons.uslci.fy24.q1.01] Vinyl ester resin; at plant [kg]
 [01] [lcacommons.uslci.fy24.q1.01] Polyvinyl chloride, PVC, resin; at plant [kg]
 [02] [lcacommons.uslci.fy24.q1.01] Polyvinyl chloride, bulk polymerised - GLO  [kg]
 [03] [lcacommons.uslci.fy24.q1.01] Polyvinylidene fluoride [kg]
 [04] [lcacommons.uslci.fy24.q1.01] Polyvinyl chloride, PVC, pre-consumer waste, for primary recycling [kg]
 [05] [lcacommons.uslci.fy24.q1.01] Vinyl acetylene [kg]
 [06] [lcacommons.uslci.fy24.q1.01] Vinyl chloride [kg]
 [07] [lcacommons.uslci.fy24.q1.01] Polyvinyl Acetate [kg]
 [08] [lcacommons.uslci.fy24.q1.01] Vinyl chloride [kg]
 [09] [lcacommons.uslci.fy24.q1.01] Vinyltoluene [kg]
 [10] [lcacommons.uslci.fy24.q1.01] Polyvinyl chloride, PVC, waste [kg]
 [11] [lcacommons.uslci.fy24.q1.01] Vinyl acetate, at plant [kg]
 [12] [lcacommons.uslci.fy24.q1.01] Polyvinyl chloride, PVC, post-consumer, for secondary recycli

In [6]:
tgts = enum(pvcs[0].targets())

GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/259a0eef-9855-4671-9b2d-7242326b3e6b/targets.. 200 [0.05 sec]
 [00] [lcacommons.uslci.fy24.q1.01] Vinyl ester resin; at plant [United States]


In [7]:
fg = cat.create_foreground('hybrid')

QQQQQQQQQQQQQQQQQQ hybrid QQQQQQQQQQQQQQQQQQ
hybrid: hybrid
hybrid: Setting NSUUID (None) None
Loading hybrid
0 total fragment entities added (0 new)


In [8]:
pvc_model = fg.create_process_model(tgts[0])

GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/5fea6b91-f71b-4a8f-864f-b64af87e8b75/exchanges/259a0eef-9855-4671-9b2d-7242326b3e6b.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/contexts/3252: Resin, Synthetic Rubber, and Artificial and Synthetic Fibers and Filaments Manufacturing.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/contexts/31-33: Manufacturing.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/contexts/Technosphere Flows.. 200 [0.05 sec]


In [9]:
fg.extend_process(pvc_model)

GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/5fea6b91-f71b-4a8f-864f-b64af87e8b75/259a0eef-9855-4671-9b2d-7242326b3e6b/inventory.. 200 [0.09 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/contexts/CUTOFF Flows.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/contexts/from water.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/contexts/to air.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/f6811440-ee37-11de-8a39-0800200c9a66.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/dce702f1-03a1-3fe9-bb48-94a5217fd8cb.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/contexts/4841: General Freight Trucking.. 200 [0.06 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/contexts/48-49: Transportation and Warehousing.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/2daa9b57-fd08-4bdf-9e15-2ec7a432aa70.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.uslc

In [10]:
pvc_model.show_tree()

   -<--O   76ad1 [       1 kg] Vinyl ester resin; at plant
    [   1 unit] Vinyl ester resin; at plant
       |        Stage: Vinyl ester resin; at plant
       | -<--*   43445 (=      1 kg) Vinyl ester resin; at plant
       |  [   1 kg] Vinyl ester resin; at plant
       |     |        Stage: CUTOFF Flows
       |     | -<----: 0d007 [   0.079 kg] Bisphenol A, at plant
       |     | -<----: ffb3e [   0.129 kg] Acrylic acid, at plant
       |     | -<----: cadce [  0.0252 MJ] Electricity, from renewable source, unspecified
       |     | -<----: 4c697 [   0.005 kg] Silica sand, at plant
       |     | =>=---: 4fc40 [ 0.00064 kg] Disposal, solid waste, to incineration with energy recovery
       |     | =>=---: 41552 [   2e-05 kg] Disposal, hazardous waste, to incineration without energy recovery
       |     | =>=---: ad7a0 [ 0.00437 kg] Disposal, solid waste, process, to sanitary landfill
       |     | =>=---: 55852 [ 0.00166 kg] Recycling, solid waste, to recycling or reuse
      

In [11]:
_=enum(pvc_model.cutoffs())

 [00] [ Vinyl ester resin; at plant ] <--  0.129 (kg) Acrylic acid, at plant  (cutoff)
 [01] [ Vinyl ester resin; at plant ] <--  0.079 (kg) Bisphenol A, at plant  (cutoff)
 [02] [ Vinyl ester resin; at plant ] <--  0.0252 (MJ) Electricity, from renewable source, unspecified  (cutoff)
 [03] [ Vinyl ester resin; at plant ] <--  0.005 (kg) Silica sand, at plant  (cutoff)
 [04] [ Vinyl ester resin; at plant ]*==>  1 (kg) Vinyl ester resin; at plant 
 [05] [ Vinyl ester resin; at plant ] ==>  0.00437 (kg) Disposal, solid waste, process, to sanitary landfill  (cutoff)
 [06] [ Vinyl ester resin; at plant ] ==>  0.00225 (kg) Disposal, hazardous waste, to incineration with energy recovery  (cutoff)
 [07] [ Vinyl ester resin; at plant ] ==>  0.00166 (kg) Recycling, solid waste, to recycling or reuse  (cutoff)
 [08] [ Vinyl ester resin; at plant ] ==>  0.00064 (kg) Disposal, solid waste, to incineration with energy recovery  (cutoff)
 [09] [ Vinyl ester resin; at plant ] ==>  0.00016 (kg) Dispos

## Part 1a - GWP

In [12]:
gwp = cat.query(r_traci[0].origin).get('Global Warming Air')

QQQQQQQQQQQQQQQQQQ lcia.traci.2.1 QQQQQQQQQQQQQQQQQQ
GET https://bk.vault.lc/origins.. 200 [0.18 sec]
lcia.traci.2.1: https://bk.vault.lc/
lcia.traci.2.1: Setting NSUUID (False) None
GET https://bk.vault.lc/lcia.traci.2.1/config.. 200 [0.05 sec]
Applying configuration to XdbClient with 0 entities at https://bk.vault.lc/
Applying context hint lcia.traci.2.1:water => to water
Applying context hint lcia.traci.2.1:air => to air
GET https://bk.vault.lc/lcia.traci.2.1/Global Warming Air.. 200 [0.05 sec]


In [13]:
baseline_result = pvc_model.fragment_lcia(gwp)
baseline_result.show_components()

POST https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/5fea6b91-f71b-4a8f-864f-b64af87e8b75/259a0eef-9855-4671-9b2d-7242326b3e6b/lcia/Global Warming Air.. 200 [0.06 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/dce702f1-03a1-3fe9-bb48-94a5217fd8cb/2afb4e2f-b224-3900-956f-03e3f27b276b/lcia/Global Warming Air.. 200 [0.12 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/2daa9b57-fd08-4bdf-9e15-2ec7a432aa70/32ed714f-405b-4678-9589-c5c0b5e055d4/lcia/Global Warming Air.. 200 [0.08 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/759a9b22-1de5-49fe-ac1d-0c287b2ef93e/ea8515c5-e3d4-4f42-a856-67a9b85b30a0/lcia/Global Warming Air.. 200 [0.08 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/89389d98-1ba6-30c5-9c33-92443694936b/06581fb2-1de0-3e78-8298-f37605dea142/lcia/Global Warming Air.. 200 [0.08 sec]
GET https://bk.vault.lc/lcacommons.uslci.fy24.q1.01/1e2429d8-8d91-328f-85c3-18d39bdebfa7/0e63d1db-e383-3c9a-b02a-b5eebafe87d1/lcia/Global Warming Air.. 200 [0.09 se

# Part 2 - What are we going to do about that acrylic acid?
seems pretty important

In [14]:
q_ee = cat.query(r_useeio[0].origin)

QQQQQQQQQQQQQQQQQQ lcacommons.useeio.2.0.1 QQQQQQQQQQQQQQQQQQ
GET https://bk.vault.lc/origins.. 200 [0.19 sec]
lcacommons.useeio.2.0.1: https://bk.vault.lc/
lcacommons.useeio.2.0.1: Setting NSUUID (False) None
GET https://bk.vault.lc/lcacommons.useeio.2.0.1/config.. 200 [0.05 sec]
Applying configuration to XdbClient with 0 entities at https://bk.vault.lc/
Applying context hint lcacommons.useeio.2.0.1:water => to water
Applying context hint lcacommons.useeio.2.0.1:air => to air


In [15]:
acr = enum(q_ee.processes(name='chem'))

GET https://bk.vault.lc/lcacommons.useeio.2.0.1/processes.. 200 [0.05 sec]
 [00] [lcacommons.useeio.2.0.1] Other basic inorganic chemicals [United States]
 [01] [lcacommons.useeio.2.0.1] Other basic organic chemicals [United States]
 [02] [lcacommons.useeio.2.0.1] Chemicals (except basic chemicals, agrichemicals, polymers, paints, pharmaceuticals,soaps, cleaning compounds) [United States]
 [03] [lcacommons.useeio.2.0.1] Petrochemicals [United States]


In [16]:
acr[1].show()

ProcessRef catalog reference (22ca518f-918e-3f1e-8f05-9997baa9acac)
origin: lcacommons.useeio.2.0.1
   UUID: 22ca518f-918e-3f1e-8f05-9997baa9acac
   Name: Other basic organic chemicals
Comment: 
==Local Fields==
    SpatialScope: United States
   TemporalScope: {'begin': '2021-01-01T00:00:00', 'end': '2021-12-31T00:00:00'}
blackbook_origin: lcacommons.useeio.2.0.1


In [17]:
acrylic_acid_proxy = acr[1]

In [18]:
acrylic_acid_usd_2012 = acrylic_acid_proxy.reference().flow
acrylic_acid_usd_2012.show()

GET https://bk.vault.lc/lcacommons.useeio.2.0.1/22ca518f-918e-3f1e-8f05-9997baa9acac/references.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.useeio.2.0.1/3bf53920-157c-4c2f-bddd-7c92c9d35f10.. 200 [0.05 sec]
FlowRef catalog reference (73c55a19-ecdc-3918-8dcb-8e6f7f1bf9bd)
origin: lcacommons.useeio.2.0.1
GET https://bk.vault.lc/lcacommons.useeio.2.0.1/contexts/3251: Basic Chemical Manufacturing.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.useeio.2.0.1/contexts/31-33: Manufacturing.. 200 [0.05 sec]
GET https://bk.vault.lc/lcacommons.useeio.2.0.1/contexts/Technosphere Flows.. 200 [0.05 sec]
Context: 3251: Basic Chemical Manufacturing
 Locale: GLO
   UUID: 73c55a19-ecdc-3918-8dcb-8e6f7f1bf9bd
   Name: Other basic organic chemicals
GET https://bk.vault.lc/lcacommons.useeio.2.0.1/73c55a19-ecdc-3918-8dcb-8e6f7f1bf9bd/doc/Comment.. 200 [0.05 sec]
Comment: 
referenceQuantity: [lcacommons.useeio.2.0.1] Producer price, USD 2012 [USD]
==Local Fields==
            unit: USD
        Sy

## 2b Build out our model 
We use *observations* of the world that we can document

In [20]:
cutoff_flow = fg.frag('ffb3e')

In [21]:
cutoff_flow.show()

(4344515) -<- ffb3e -<- ---:  [--] Acrylic acid, at plant 
LcFragment Entity (ref ffb3ea47-6bef-4ba9-86f5-5da41f6fde5a)
origin: hybrid
reference: (76ad11f) -<- 43445 -<- -*    [   1 kg] Vinyl ester resin; at plant 
     Name: Acrylic acid, at plant
  Comment: Proxy for methylacrylic acid
StageName: CUTOFF Flows
Exchange values: 
              Cached: 0.129
            Observed: 0.129

Balance flow: False
Terminations: 
            Scenario  Termination
                None: ---:       None


### Step 1: convert kg to USD 2024

In [19]:
usd_2024 = fg.new_quantity('USD 2024', ref_unit='USD 2024')

In [22]:
acr_dollars = fg.new_flow('Acrylic acid, 2024 USD per kg', ref_quantity=usd_2024)

In [23]:
acr_conv_1 = fg.new_fragment(acr_dollars, 'Input', parent=cutoff_flow)

In [24]:
pvc_model.show_tree()

   -<--O   76ad1 [       1 kg] Vinyl ester resin; at plant
    [   1 unit] Vinyl ester resin; at plant
       |        Stage: Vinyl ester resin; at plant
       | -<--*   43445 (=      1 kg) Vinyl ester resin; at plant
       |  [   1 kg] Vinyl ester resin; at plant
       |     |        Stage: CUTOFF Flows
       |     | -<----: 0d007 [   0.079 kg] Bisphenol A, at plant
       |     | -<----: cadce [  0.0252 MJ] Electricity, from renewable source, unspecified
       |     | -<----: 4c697 [   0.005 kg] Silica sand, at plant
       |     | =>=---: 4fc40 [ 0.00064 kg] Disposal, solid waste, to incineration with energy recovery
       |     | =>=---: 41552 [   2e-05 kg] Disposal, hazardous waste, to incineration without energy recovery
       |     | =>=---: ad7a0 [ 0.00437 kg] Disposal, solid waste, process, to sanitary landfill
       |     | =>=---: 55852 [ 0.00166 kg] Recycling, solid waste, to recycling or reuse
       |     | =>=---: c7a37 [ 0.00016 kg] Disposal, solid waste, to inc

The cutoff now shows the same amount, but it's reported in units of USD

In [25]:
_=enum(pvc_model.cutoffs())

 [00] [ Vinyl ester resin; at plant ] <--  0.129 (USD 2024) Acrylic acid, 2024 USD per kg  (cutoff)
 [01] [ Vinyl ester resin; at plant ] <--  0.079 (kg) Bisphenol A, at plant  (cutoff)
 [02] [ Vinyl ester resin; at plant ] <--  0.0252 (MJ) Electricity, from renewable source, unspecified  (cutoff)
 [03] [ Vinyl ester resin; at plant ] <--  0.005 (kg) Silica sand, at plant  (cutoff)
 [04] [ Vinyl ester resin; at plant ]*==>  1 (kg) Vinyl ester resin; at plant 
 [05] [ Vinyl ester resin; at plant ] ==>  0.00437 (kg) Disposal, solid waste, process, to sanitary landfill  (cutoff)
 [06] [ Vinyl ester resin; at plant ] ==>  0.00225 (kg) Disposal, hazardous waste, to incineration with energy recovery  (cutoff)
 [07] [ Vinyl ester resin; at plant ] ==>  0.00166 (kg) Recycling, solid waste, to recycling or reuse  (cutoff)
 [08] [ Vinyl ester resin; at plant ] ==>  0.00064 (kg) Disposal, solid waste, to incineration with energy recovery  (cutoff)
 [09] [ Vinyl ester resin; at plant ] ==>  0.0001

We want to set the price of acrylic acid - something we can *observe*

In [26]:
acr_conv_1.observe(15)  # USD 2024 per kg

In [30]:
_=enum(pvc_model.cutoffs())

 [00] [ Vinyl ester resin; at plant ] <--  0.129 (USD 2024) Acrylic acid, 2024 USD per kg  (cutoff)
 [01] [ Vinyl ester resin; at plant ] <--  0.079 (kg) Bisphenol A, at plant  (cutoff)
 [02] [ Vinyl ester resin; at plant ] <--  0.0252 (MJ) Electricity, from renewable source, unspecified  (cutoff)
 [03] [ Vinyl ester resin; at plant ] <--  0.005 (kg) Silica sand, at plant  (cutoff)
 [04] [ Vinyl ester resin; at plant ]*==>  1 (kg) Vinyl ester resin; at plant 
 [05] [ Vinyl ester resin; at plant ] ==>  0.00437 (kg) Disposal, solid waste, process, to sanitary landfill  (cutoff)
 [06] [ Vinyl ester resin; at plant ] ==>  0.00225 (kg) Disposal, hazardous waste, to incineration with energy recovery  (cutoff)
 [07] [ Vinyl ester resin; at plant ] ==>  0.00166 (kg) Recycling, solid waste, to recycling or reuse  (cutoff)
 [08] [ Vinyl ester resin; at plant ] ==>  0.00064 (kg) Disposal, solid waste, to incineration with energy recovery  (cutoff)
 [09] [ Vinyl ester resin; at plant ] ==>  0.0001

#### Hold on
Why didn't it change??

Because Antelope fragments distinguish between a flow's "default" value and its "observed" value.  When you run LCIA, it always uses observed values- but during model construction default values are shown.

We can specify to run the *observed* numbers, however:

In [29]:
_=enum(pvc_model.cutoffs(observed=True))

 [00] [ Vinyl ester resin; at plant ] <--  1.94 (USD 2024) Acrylic acid, 2024 USD per kg  (cutoff)
 [01] [ Vinyl ester resin; at plant ] <--  0.079 (kg) Bisphenol A, at plant  (cutoff)
 [02] [ Vinyl ester resin; at plant ] <--  0.0252 (MJ) Electricity, from renewable source, unspecified  (cutoff)
 [03] [ Vinyl ester resin; at plant ] <--  0.005 (kg) Silica sand, at plant  (cutoff)
 [04] [ Vinyl ester resin; at plant ]*==>  1 (kg) Vinyl ester resin; at plant 
 [05] [ Vinyl ester resin; at plant ] ==>  0.00437 (kg) Disposal, solid waste, process, to sanitary landfill  (cutoff)
 [06] [ Vinyl ester resin; at plant ] ==>  0.00225 (kg) Disposal, hazardous waste, to incineration with energy recovery  (cutoff)
 [07] [ Vinyl ester resin; at plant ] ==>  0.00166 (kg) Recycling, solid waste, to recycling or reuse  (cutoff)
 [08] [ Vinyl ester resin; at plant ] ==>  0.00064 (kg) Disposal, solid waste, to incineration with energy recovery  (cutoff)
 [09] [ Vinyl ester resin; at plant ] ==>  0.00016

In [33]:
acr_conv_1.show()

(ffb3ea4) -<- d6d1a -<- ---:  [--] Acrylic acid, 2024 USD per kg 
LcFragment Entity (ref d6d1a3e7-79be-40b3-91b6-422424301bf3)
origin: hybrid
reference: (4344515) -<- ffb3e -<- -O    [   1 unit] Acrylic acid, at plant 
     Name: Acrylic acid, 2024 USD per kg
  Comment: 
StageName: 
Exchange values: 
              Cached: 15
            Observed: 15

Balance flow: False
Terminations: 
            Scenario  Termination
                None: ---:       None


In [32]:
acr_conv_1.cached_ev = acr_conv_1.observed_ev

In [ ]:
acr_conv_1.show()

### Step 2 - Convert 2024 USD to 2012 USD
This is a fixed value, so we can "characterize" our flow

We don't need to make a new quantity in this case because we already have one

In [34]:
usd_2012 = acrylic_acid_usd_2012.reference_entity

In [36]:
acr_dollars.characterize(usd_2012, 0.57)

Characterization(Acrylic acid, 2024 USD per kg, USD 2024, None:   0.57 (USD))

In [37]:
acr_conv_2 = fg.new_fragment(acrylic_acid_usd_2012, 'Input', parent=acr_conv_1, balance=True)

In [38]:
pvc_model.show_tree()

   -<--O   76ad1 [       1 kg] Vinyl ester resin; at plant
    [   1 unit] Vinyl ester resin; at plant
       |        Stage: Vinyl ester resin; at plant
       | -<--*   43445 (=      1 kg) Vinyl ester resin; at plant
       |  [   1 kg] Vinyl ester resin; at plant
       |     |        Stage: CUTOFF Flows
       |     | -<----: 0d007 [   0.079 kg] Bisphenol A, at plant
       |     | -<----: cadce [  0.0252 MJ] Electricity, from renewable source, unspecified
       |     | -<----: 4c697 [   0.005 kg] Silica sand, at plant
       |     | =>=---: 4fc40 [ 0.00064 kg] Disposal, solid waste, to incineration with energy recovery
       |     | =>=---: 41552 [   2e-05 kg] Disposal, hazardous waste, to incineration without energy recovery
       |     | =>=---: ad7a0 [ 0.00437 kg] Disposal, solid waste, process, to sanitary landfill
       |     | =>=---: 55852 [ 0.00166 kg] Recycling, solid waste, to recycling or reuse
       |     | =>=---: c7a37 [ 0.00016 kg] Disposal, solid waste, to inc

In [39]:
_=enum(pvc_model.cutoffs(observed=True))

 [00] [ Vinyl ester resin; at plant ] <--  1.1 (USD) Other basic organic chemicals  (cutoff)
 [01] [ Vinyl ester resin; at plant ] <--  0.079 (kg) Bisphenol A, at plant  (cutoff)
 [02] [ Vinyl ester resin; at plant ] <--  0.0252 (MJ) Electricity, from renewable source, unspecified  (cutoff)
 [03] [ Vinyl ester resin; at plant ] <--  0.005 (kg) Silica sand, at plant  (cutoff)
 [04] [ Vinyl ester resin; at plant ]*==>  1 (kg) Vinyl ester resin; at plant 
 [05] [ Vinyl ester resin; at plant ] ==>  0.00437 (kg) Disposal, solid waste, process, to sanitary landfill  (cutoff)
 [06] [ Vinyl ester resin; at plant ] ==>  0.00225 (kg) Disposal, hazardous waste, to incineration with energy recovery  (cutoff)
 [07] [ Vinyl ester resin; at plant ] ==>  0.00166 (kg) Recycling, solid waste, to recycling or reuse  (cutoff)
 [08] [ Vinyl ester resin; at plant ] ==>  0.00064 (kg) Disposal, solid waste, to incineration with energy recovery  (cutoff)
 [09] [ Vinyl ester resin; at plant ] ==>  0.00016 (kg) 

In [42]:
acr_conv_2.terminate(acrylic_acid_proxy)

In [43]:
_=enum(pvc_model.cutoffs(observed=True))

 [00] [ Vinyl ester resin; at plant ] <--  0.079 (kg) Bisphenol A, at plant  (cutoff)
 [01] [ Vinyl ester resin; at plant ] <--  0.0252 (MJ) Electricity, from renewable source, unspecified  (cutoff)
 [02] [ Vinyl ester resin; at plant ] <--  0.005 (kg) Silica sand, at plant  (cutoff)
 [03] [ Vinyl ester resin; at plant ]*==>  1 (kg) Vinyl ester resin; at plant 
 [04] [ Vinyl ester resin; at plant ] ==>  0.00437 (kg) Disposal, solid waste, process, to sanitary landfill  (cutoff)
 [05] [ Vinyl ester resin; at plant ] ==>  0.00225 (kg) Disposal, hazardous waste, to incineration with energy recovery  (cutoff)
 [06] [ Vinyl ester resin; at plant ] ==>  0.00166 (kg) Recycling, solid waste, to recycling or reuse  (cutoff)
 [07] [ Vinyl ester resin; at plant ] ==>  0.00064 (kg) Disposal, solid waste, to incineration with energy recovery  (cutoff)
 [08] [ Vinyl ester resin; at plant ] ==>  0.00016 (kg) Disposal, solid waste, to incineration without energy recovery  (cutoff)
 [09] [ Vinyl ester 

In [44]:
pvc_model.show_tree()

   -<--O   76ad1 [       1 kg] Vinyl ester resin; at plant
    [   1 unit] Vinyl ester resin; at plant
       |        Stage: Vinyl ester resin; at plant
       | -<--*   43445 (=      1 kg) Vinyl ester resin; at plant
       |  [   1 kg] Vinyl ester resin; at plant
       |     |        Stage: CUTOFF Flows
       |     | -<----: 0d007 [   0.079 kg] Bisphenol A, at plant
       |     | -<----: cadce [  0.0252 MJ] Electricity, from renewable source, unspecified
       |     | -<----: 4c697 [   0.005 kg] Silica sand, at plant
       |     | =>=---: 4fc40 [ 0.00064 kg] Disposal, solid waste, to incineration with energy recovery
       |     | =>=---: 41552 [   2e-05 kg] Disposal, hazardous waste, to incineration without energy recovery
       |     | =>=---: ad7a0 [ 0.00437 kg] Disposal, solid waste, process, to sanitary landfill
       |     | =>=---: 55852 [ 0.00166 kg] Recycling, solid waste, to recycling or reuse
       |     | =>=---: c7a37 [ 0.00016 kg] Disposal, solid waste, to inc

In [45]:
pvc_model.fragment_lcia(gwp).show_components()

GET https://bk.vault.lc/lcacommons.useeio.2.0.1/22ca518f-918e-3f1e-8f05-9997baa9acac/73c55a19-ecdc-3918-8dcb-8e6f7f1bf9bd/lcia/Global Warming Air.. 200 [0.26 sec]
[lcia.traci.2.1] Global Warming Air [kg CO2 eq] [TRACI 2.1] kg CO2 eq
------------------------------------------------------------
S      3.05 =      0.428 x       7.13 | 2ee08       0.428 [ Input] -B*  Epoxy; at plant [United States]
S      1.81 =        1.1 x       1.64 | 4ff0f         1.1 [ Input] -B*  Other basic organic chemicals [United States]
S      1.16 =      0.364 x       3.18 | 770da       0.364 [ Input] -B*  Styrene; at plant [Northern America]
S     0.132 =      0.054 x       2.44 | 80542       0.054 [ Input] -B*  Natural gas, combusted in industrial boiler [Northern America]
S     0.101 =      0.677 x       0.15 | 79108       0.677 [ Input] -B*  Electricity, at Grid, US, 2010 [Northern America]
S      0.06 =       2.96 x     0.0203 | 884ea        2.96 [ Input] -B*  Transport, ocean freighter, average fuel mix [

## 3 But wait, there's more
We can also use observations to set scenarios

### Step 1 - change market price of acrylic acid

In [47]:
acr_conv_1.observe(45, scenario='high inflation')

In [48]:
acr_conv_1.show()

(ffb3ea4) -<- d6d1a -<- -O    [   1 unit] Acrylic acid, 2024 USD per kg 
LcFragment Entity (ref d6d1a3e7-79be-40b3-91b6-422424301bf3)
origin: hybrid
reference: (4344515) -<- ffb3e -<- -O    [   1 unit] Acrylic acid, at plant 
     Name: Acrylic acid, 2024 USD per kg
  Comment: 
StageName: 
Exchange values: 
              Cached: 15
            Observed: 15
      high inflation: 45

Balance flow: False
Terminations: 
            Scenario  Termination
                None: -O   Foreground


In [49]:
pvc_model.fragment_lcia(gwp, scenario='high inflation').show_components()

[lcia.traci.2.1] Global Warming Air [kg CO2 eq] [TRACI 2.1] kg CO2 eq
------------------------------------------------------------
S      5.42 =       3.31 x       1.64 | 4ff0f        3.31 [ Input] -B*  Other basic organic chemicals [United States]
S      3.05 =      0.428 x       7.13 | 2ee08       0.428 [ Input] -B*  Epoxy; at plant [United States]
S      1.16 =      0.364 x       3.18 | 770da       0.364 [ Input] -B*  Styrene; at plant [Northern America]
S     0.132 =      0.054 x       2.44 | 80542       0.054 [ Input] -B*  Natural gas, combusted in industrial boiler [Northern America]
S     0.101 =      0.677 x       0.15 | 79108       0.677 [ Input] -B*  Electricity, at Grid, US, 2010 [Northern America]
S      0.06 =       2.96 x     0.0203 | 884ea        2.96 [ Input] -B*  Transport, ocean freighter, average fuel mix [United States]
S    0.0495 =      0.483 x      0.103 | 66c38       0.483 [ Input] -B*  Transport, combination truck, average fuel mix [Northern America]
S    0.019

### Step 2 - change proxy model for acrylic acid

In [50]:
_=enum(acr)

 [00] [lcacommons.useeio.2.0.1] Other basic inorganic chemicals [United States]
 [01] [lcacommons.useeio.2.0.1] Other basic organic chemicals [United States]
 [02] [lcacommons.useeio.2.0.1] Chemicals (except basic chemicals, agrichemicals, polymers, paints, pharmaceuticals,soaps, cleaning compounds) [United States]
 [03] [lcacommons.useeio.2.0.1] Petrochemicals [United States]


In [51]:
acr_conv_2.show()

(d6d1a3e) -<- 4ff0f -<- -B*   [   1 USD] Other basic organic chemicals 
LcFragment Entity (ref 4ff0f4c8-7bf4-4469-935d-04caaaaf1f8b)
origin: hybrid
reference: (ffb3ea4) -<- d6d1a -<- -O    [   1 unit] Acrylic acid, 2024 USD per kg 
     Name: Other basic organic chemicals
  Comment: 
StageName: Other basic organic chemicals
Exchange values: 
              Cached: 1
            Observed: 0

Balance flow: True ([lcacommons.useeio.2.0.1] Producer price, USD 2012 [USD])
Terminations: 
            Scenario  Termination
                None: -B*        [lcacommons.useeio.2.0.1] Other basic organic chemicals [United States]


In [52]:
acr_conv_2.terminate(acr[3], scenario='petro proxy')

GET https://bk.vault.lc/lcacommons.useeio.2.0.1/e2d57345-e81e-39ce-a69f-3899e31c6f3f/references.. 200 [0.21 sec]


In [53]:
pvc_model.fragment_lcia(gwp, scenario='petro proxy').show_components()

GET https://bk.vault.lc/lcacommons.useeio.2.0.1/e2d57345-e81e-39ce-a69f-3899e31c6f3f/b42091c0-2aeb-38b6-b243-33bf8b71273f/lcia/Global Warming Air.. 200 [0.30 sec]
[lcia.traci.2.1] Global Warming Air [kg CO2 eq] [TRACI 2.1] kg CO2 eq
------------------------------------------------------------
S      3.05 =      0.428 x       7.13 | 2ee08       0.428 [ Input] -B*  Epoxy; at plant [United States]
S      1.16 =      0.364 x       3.18 | 770da       0.364 [ Input] -B*  Styrene; at plant [Northern America]
S      1.03 =        1.1 x      0.937 | 4ff0f         1.1 [ Input] -B*  Petrochemicals [United States]
S     0.132 =      0.054 x       2.44 | 80542       0.054 [ Input] -B*  Natural gas, combusted in industrial boiler [Northern America]
S     0.101 =      0.677 x       0.15 | 79108       0.677 [ Input] -B*  Electricity, at Grid, US, 2010 [Northern America]
S      0.06 =       2.96 x     0.0203 | 884ea        2.96 [ Input] -B*  Transport, ocean freighter, average fuel mix [United States]


#### mix scenarios together

In [55]:
pvc_model.fragment_lcia(gwp, scenario=('high inflation', 'petro proxy')).show_components()

[lcia.traci.2.1] Global Warming Air [kg CO2 eq] [TRACI 2.1] kg CO2 eq
------------------------------------------------------------
S       3.1 =       3.31 x      0.937 | 4ff0f        3.31 [ Input] -B*  Petrochemicals [United States]
S      3.05 =      0.428 x       7.13 | 2ee08       0.428 [ Input] -B*  Epoxy; at plant [United States]
S      1.16 =      0.364 x       3.18 | 770da       0.364 [ Input] -B*  Styrene; at plant [Northern America]
S     0.132 =      0.054 x       2.44 | 80542       0.054 [ Input] -B*  Natural gas, combusted in industrial boiler [Northern America]
S     0.101 =      0.677 x       0.15 | 79108       0.677 [ Input] -B*  Electricity, at Grid, US, 2010 [Northern America]
S      0.06 =       2.96 x     0.0203 | 884ea        2.96 [ Input] -B*  Transport, ocean freighter, average fuel mix [United States]
S    0.0495 =      0.483 x      0.103 | 66c38       0.483 [ Input] -B*  Transport, combination truck, average fuel mix [Northern America]
S    0.0198 =      0.813 